In [ ]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 16.4 MB/s eta 0:00:00


In [ ]:
import pymongo
from pymongo import MongoClient
import os

# connection to MongoDB

In [ ]:
from pymongo.mongo_client import MongoClient

uri = "mongodb+srv://haya6000:<pass>@cluster3.qlt9k1t.mongodb.net/?retryWrites=true&w=majority&appName=Cluster3"

# Create a new client and connect to the server
client = MongoClient(uri)

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)



# Create a database
db = client['traffic_db']
# Create a collection
traffic_collection = db['traffic_data']


Pinged your deployment. You successfully connected to MongoDB!


In [ ]:
traffic_collection.insert_many([
{'road': 'Highway A1', 'date': '2024-07-10', 'time': '07:30', 'traffic_volume': 1100},
{'road': 'Highway A2', 'date': '2024-07-11', 'time': '08:30', 'traffic_volume': 900},
{'road': 'Highway A1', 'date': '2024-07-11', 'time': '08:00', 'traffic_volume': 1200,
'weather_condition': 'Clear', 'vehicle_type_breakdown': {'car': 800, 'truck': 300, 'motorcycle': 100}},
{'road': 'Highway A3', 'date': '2024-07-12', 'time': '09:00', 'traffic_volume': 800},
{'road': 'Highway A1', 'date': '2024-07-11', 'time': '10:00', 'traffic_volume': 1300}
])

InsertManyResult([ObjectId('66afb51dc54ad417f2d8fdce'), ObjectId('66afb51dc54ad417f2d8fdcf'), ObjectId('66afb51dc54ad417f2d8fdd0'), ObjectId('66afb51dc54ad417f2d8fdd1'), ObjectId('66afb51dc54ad417f2d8fdd2')], acknowledged=True)

# Data insertion

In [ ]:
traffic_collection.insert_one({'road': 'Highway A1' , 'date' : '2024-07-11' , 'time' : '08:00' , 'traffic_volume': 1200})
traffic_collection.insert_one({'road': 'Highway A1' , 'date' : '2024-07-11' , 'time' : '09:00' , 'traffic_volume': 1100,
                               'weather_condition': 'Clear', 'vehicle_type_breakdown': {'car': 800, 'truck': 300, 'motorcycle':100}
                               })

InsertOneResult(ObjectId('66afb948c54ad417f2d8fdd6'), acknowledged=True)

# Query Operation

In [ ]:
find_traffic = traffic_collection.find({'traffic_volume': {'$gt': 1000, '$lt': 1500}})

for val in find_traffic:
    print(val)
    print(f'road: {val["road"]}')
    print(f'date: {val["date"]}')
    print(f'time: {val["time"]}')
    print(f'traffic_volume: {val["traffic_volume"]}')

    try:
        print(f'weather_condition: {val["weather_condition"]}')
    except KeyError:
        print('weather_condition: Not available')

    try:
        print(f'vehicle_type_breakdown: {val["vehicle_type_breakdown"]}')
    except KeyError:
        print('vehicle_type_breakdown: Not available')

    print('________________________________________')



{'_id': ObjectId('66afb51dc54ad417f2d8fdce'), 'road': 'Highway A1', 'date': '2024-07-10', 'time': '07:30', 'traffic_volume': 1100}
road: Highway A1
date: 2024-07-10
time: 07:30
traffic_volume: 1100
weather_condition: Not available
vehicle_type_breakdown: Not available
________________________________________
{'_id': ObjectId('66afb51dc54ad417f2d8fdd0'), 'road': 'Highway A1', 'date': '2024-07-11', 'time': '08:00', 'traffic_volume': 1200, 'weather_condition': 'Clear', 'vehicle_type_breakdown': {'car': 800, 'truck': 300, 'motorcycle': 100}}
road: Highway A1
date: 2024-07-11
time: 08:00
traffic_volume: 1200
weather_condition: Clear
vehicle_type_breakdown: {'car': 800, 'truck': 300, 'motorcycle': 100}
________________________________________
{'_id': ObjectId('66afb51dc54ad417f2d8fdd2'), 'road': 'Highway A1', 'date': '2024-07-11', 'time': '10:00', 'traffic_volume': 1300}
road: Highway A1
date: 2024-07-11
time: 10:00
traffic_volume: 1300
weather_condition: Not available
vehicle_type_breakdown

# Update operations

In [ ]:
# Increment the traffic volume by 200 vehicles for all documents where the road is "Highway A1" and the time is between "08:00" and "10:00" on "2024-
# 07-11".
traffic_collection.update_many(
    {'road': 'Highway A1', 'time': {'$gte': '08:00', '$lte': '10:00'}, 'date': '2024-07-1'},
    {'$inc': {'traffic_volume': 200}}
)

# Update the vehicle type breakdown to {"car": 850, "truck": 320, "motorcycle": 130} for the document with the highest traffic volume.
highest_traffic_doc = traffic_collection.find_one({}, sort=[('traffic_volume', -1)], limit=1)

update_query = {'$set': {'vehicle_type_breakdown': {'car': 850, 'truck': 320, 'motorcycle': 130}}}

result = traffic_collection.update_one(
    {'_id': highest_traffic_doc['_id']},
    update_query
)

# Delete operations

In [ ]:
traffic_collection.delete_one({'weather_condition': 'Rainy'})

filter_query = {
    'traffic_volume': {'$lt': 500},
    'date': {'$lt': '2024-07-11'}
}
traffic_collection.delete_many(filter_query)

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff0000000000000064'), 'opTime': {'ts': Timestamp(1722793331, 3), 't': 100}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1722793331, 3), 'signature': {'hash': b'/\x83|\xbd\xcc^\x0eV<N\x8eq\xcc\xdd\xfd\x16\x96-\xba\xb0', 'keyId': 7343326493319102465}}, 'operationTime': Timestamp(1722793331, 3)}, acknowledged=True)